In [5]:
import pandas as pd
import tabpy_client
from sklearn.model_selection import train_test_split
import numpy as np

In [21]:
avg_milage=pd.read_csv("avg_milage.csv")
riskfactor= pd.read_csv("riskfactor.csv")
geolocation= pd.read_csv("geolocation.csv")
idling=pd.read_csv("count_idling.csv")
truck_milage= pd.read_csv("truck_milage.csv")

In [22]:
geolocation

,truckid,driverid,event,latitude,longitude,city,state,velocity,event_ind,idling_ind
0,A54,A54,normal,38.440467,-122.714431,Santa Rosa,California,17,0,0
1,A20,A20,normal,36.977173,-121.899402,Aptos,California,27,0,0
2,A40,A40,overspeed,37.957702,-121.290780,Stockton,California,77,1,0
3,A31,A31,normal,39.409608,-123.355566,Willits,California,22,0,0
4,A71,A71,normal,33.683947,-117.794694,Irvine,California,43,0,0
...,...,...,...,...,...,...,...,...,...,...
7995,A19,A19,normal,35.373292,-119.018712,Bakersfield,California,25,0,0
7996,A14,A14,normal,37.957702,-121.290780,Stockton,California,0,0,1
7997,A88,A88,normal,38.440467,-122.714431,Santa Rosa,California,0,0,1
7998,A75,A75,normal,38.364080,-122.524149,Glen Ellen,California,17,0,0


In [23]:
geo_new=geolocation.groupby("driverid",as_index=False).mean()[['driverid','velocity']]

In [24]:
avg_milage["driverid"]= avg_milage["truckid"]
avg_milage.drop("truckid",axis=1,inplace=True)

In [25]:
data1=pd.merge(avg_milage,riskfactor,left_on="driverid",right_on="driverid",how="inner")
data2=pd.merge(data1,geo_new,left_on="driverid",right_on="driverid",how="inner")
data=pd.merge(data2,idling,left_on="driverid",right_on="driverid",how="inner")
data["riskfactor"]=(data["events"]/data["totmiles"])*1000000
func= lambda x: 1 if x>7 else 0
data["logic"]=data["riskfactor"].apply(func)

In [26]:
y=data["logic"]
list_did=data["driverid"].tolist()
x=data.drop(["logic","driverid","riskfactor","events","totmiles"],axis=1)
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.40, random_state=42)

In [27]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(random_state=43,fit_intercept=True).fit(x_train, y_train)
predicted=clf.predict_proba(x_test)

D:\nickiminaj\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [28]:
clf.score(x_test,y_test)

0.45

In [29]:
# Connect to TabPy server using the client library
connection = tabpy_client.Client('http://localhost:9004/')

In [30]:
def predictProb(avgmpg,velocity,idling_count):
    x=np.column_stack([avgmpg,velocity,idling_count])
    pred=clf.predict_proba(x).tolist()
    return [x[1] for x in pred] 
    

In [31]:
# Publish the predictProb function to TabPy server so it can be used from Tableau
# Using the name modelPredict and a short description of what it does
connection.deploy('modelPredict',predictProb,
                  'Gives the probability of having a risk greater than 7',override=True)

In [32]:
x.head()

,avgmpg,velocity,idling_count
0,4.785823,37.4500,14
1,5.401718,31.7625,21
2,4.939039,30.3625,24
3,5.502369,33.8375,12
4,4.686164,35.3000,20
